In [1]:
import setup

setup.init_django()

In [2]:
from decouple import config

In [3]:
from blog.models import BlogPost
from rag import db as rag_db, settings as rag_settings

In [4]:
from sqlalchemy import (
    create_engine,
    inspect,
)

from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.retrievers import NLSQLRetriever

In [5]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.engine import reflection
from sqlalchemy.schema import CreateTable
from sqlalchemy.sql import select
from decouple import config

# Create engines
neon_engine = create_engine(config("DATABASE_URL"))
vector_engine = create_engine(config("DATABASE_URL_POOL"))

neon_conn = neon_engine.connect()
vector_conn = vector_engine.connect()

In [10]:
from sqlalchemy import create_engine, MetaData, Table, text, inspect
from sqlalchemy.schema import CreateTable
from sqlalchemy.dialects.postgresql import dialect
from sqlalchemy.sql import select
from decouple import config
from sqlalchemy.exc import CompileError

# Load database URLs from .env
source_db_url = config("DATABASE_URL")        # neondb
target_db_url = config("DATABASE_URL_POOL")   # vector_db

# Create SQLAlchemy engine for both databases
source_engine = create_engine(source_db_url)
target_engine = create_engine(target_db_url)

# Create connections
source_conn = source_engine.connect()
target_conn = target_engine.connect()

# Initialize metadata
metadata = MetaData()

# ✅ Use sqlalchemy.inspect() instead of deprecated reflection.Inspector.from_engine()
inspector = inspect(source_engine)

# Loop through each table
for table_name in inspector.get_table_names():
    print(f"Copying table: {table_name}")

    try:
        table = Table(table_name, metadata, autoload_with=source_engine)
        create_stmt = CreateTable(table).compile(dialect=dialect())
    except CompileError as e:
        print(f"Skipping table {table_name} due to unsupported type: {e}")
        continue

   
    sql_lines = str(create_stmt).splitlines()
    
    # Filter out lines that contain FOREIGN KEY constraints
    cleaned_sql_lines = [line for line in sql_lines if 'FOREIGN KEY' not in line]
    
    # Rebuild the SQL
    cleaned_sql = "\n".join(cleaned_sql_lines)
    
    # Run the cleaned SQL
    target_conn.execute(text(cleaned_sql))

print("✅ All tables copied successfully.")


Copying table: django_migrations
Copying table: auth_user_groups
Copying table: products_product
Copying table: auth_permission
Copying table: django_content_type
Copying table: auth_user
Copying table: django_admin_log
Copying table: blog_blogpost


C:\Users\MYHP\AppData\Local\Temp\ipykernel_17396\1903555549.py:31: SAWarning: Did not recognize type 'vector' of column 'embedding'
  table = Table(table_name, metadata, autoload_with=source_engine)


Skipping table blog_blogpost due to unsupported type: (in table 'blog_blogpost', column 'embedding'): Can't generate DDL for NullType(); did you forget to specify a type on this Column?
Copying table: auth_group_permissions
Copying table: auth_user_user_permissions
Copying table: django_session
Copying table: auth_group
Copying table: products_embedding
Skipping table products_embedding due to unsupported type: (in table 'products_embedding', column 'embedding'): Can't generate DDL for NullType(); did you forget to specify a type on this Column?
Copying table: analytics_pageview


ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near ")"
LINE 7: )
        ^

[SQL: 
CREATE TABLE analytics_pageview (
	id BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 NO CYCLE), 
	timestamp TIMESTAMP WITH TIME ZONE NOT NULL, 
	post_id BIGINT NOT NULL, 
	CONSTRAINT analytics_pageview_pkey PRIMARY KEY (id), 
)
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
# initialize default LlamaIndex settings
rag_settings.init()
# get pooled Neon database string from .env or env vars
vector_database_url = rag_db.get_database_url(use_pooling=True)

In [ ]:
engine = create_engine(vector_database_url)

In [ ]:
inspect(engine).get_table_names()

In [ ]:
tables = []
models = [BlogPost]
for model in models:
    table = model._meta.db_table
    tables.append(table)

In [ ]:
tables

In [ ]:
from sqlalchemy import inspect

inspector = inspect(engine)
print(inspector.get_table_names())

In [ ]:
sql_database = SQLDatabase(engine, include_tables=tables)

In [ ]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=tables,
)

In [ ]:
response = sql_query_engine.query("What is my most recent blog post id?")
response

In [ ]:
for node in response.source_nodes:
    print(node.node.get_content())

In [ ]:
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=tables, return_raw=True
)

r = nl_sql_retriever.retrieve("What is my least most recent blog post?")

In [ ]:
print(r)
for node in r:
    print(node)
    print(node.metadata)